In [1]:
import os
from selenium import webdriver #Webdriver de Selenium qui permet de contrôler un navigateur
from selenium.webdriver.common.by import By #Permet d'accéder aux différents élements de la page web
from selenium.webdriver.common.keys import Keys # Importe les clefs pour les touches du clavier
from webdriver_manager.chrome import ChromeDriverManager #Assure la gestion du webdriver de Chrome
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import math
import time
from time import sleep 

from tqdm import tqdm # librairie permettant d'indiquer le pourcentage de la boucle for


import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np
gaming = pd.read_csv('vgsales.csv', index_col=0)
gaming.head(5)


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [3]:
gaming_parts = np.array_split(gaming, 16)
gaming_parts[2].head()

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Rank,,,,,,,,,,
2078,Wheel of Fortune,Wii,2010.0,Misc,THQ,0.89,0.04,0.0,0.06,1.0
2079,Disney Infinity,PS3,2013.0,Action,Disney Interactive Studios,0.48,0.35,0.0,0.16,1.0
2080,Dark Souls,X360,2011.0,Role-Playing,Namco Bandai Games,0.64,0.28,0.0,0.08,1.0
2081,Puzzler Collection,DS,2008.0,Puzzle,Ubisoft,0.17,0.78,0.0,0.04,1.0
2082,Jampack Winter 2000,PS,2000.0,Misc,Sony Computer Entertainment,0.55,0.38,0.0,0.07,1.0


In [4]:
def scroll(value): #Définition d'une fonction pour scroller automatiquement
    for i in range(20): #nombre de micro-scroll à effectuer
        driver.execute_script("window.scrollBy(0, {})".format(value))
        time.sleep(0.1) #Temps entre chaque scroll

In [11]:
gaming_JVC=gaming_parts[2]
gaming_JVC=gaming_JVC.iloc[776:]

In [12]:
#scrapping part 1
#gaming_JVC=gaming_parts[2]

# Ouvrir une page Google Chrome
driver = webdriver.Chrome(ChromeDriverManager().install()) 
# créer une attente implicite
driver.implicitly_wait(10)

#Lancer une recherche en utilisant un fichier excel ou csv (en ittérant sur une colonne)

for i, row in tqdm(gaming_JVC.iterrows(), total = len(gaming_JVC)):
    
    # Ouvrir l'adresse du site web
    driver.get('https://www.jeuxvideo.com/tous-les-jeux/')
    try:
        # accepter les cookies
        driver.find_element(By.XPATH, "//*[@id='cmp-main']/button[2]").click()
    except:
        pass
    
    #Ajout d'un temps de deux secondes avant de lancer l'action suivante
    time.sleep(1) 
    
    loupe_recherche=driver.find_element(By.CLASS_NAME, 'searchContainer__1f9uG_')
    loupe_recherche.click()

    try:
        barre_recherche_site=driver.find_element(By.CLASS_NAME,"searchInput__38b75-")
        barre_recherche_site.send_keys(row['Name'])
        barre_recherche_site.send_keys(Keys.ENTER)

    except:
        pass

    try:    
        premier_jeu=driver.find_elements(By.CLASS_NAME,"col")[0]
        premier_jeu.click()

        # cliquer sur la bonne plateforme
        boutons_plateforme=driver.find_elements(By.CLASS_NAME,"gameHeaderBanner__platform")
        boutons_plateforme
    except:
        pass

    try:
        for bouton in boutons_plateforme:
            if str(bouton.text) == str(row['Platform']):
                bouton.click()
            else:
                pass
    except:
        pass

    try:
        test_note=driver.find_element(By.CLASS_NAME, "gameCharacteristicsMain__gauge")
        gaming_JVC.loc[i, 'test_note_JVC']=test_note.text
    except:
        gaming_JVC.loc[i, 'test_note_JVC']='no_data'
    
    try:
        avis_count=driver.find_element(By.CLASS_NAME, "gameCharacteristicsMain__count")
        gaming_JVC.loc[i, 'avis_count_JVC']=avis_count.text
    except:
        gaming_JVC.loc[i, 'avis_count_JVC']='no_data'

    try:
        avis_note=driver.find_elements(By.CLASS_NAME, "gameCharacteristicsMain__gauge")[1]
        gaming_JVC.loc[i, 'avis_note_JVC']=avis_note.text
    except:
        gaming_JVC.loc[i, 'avis_note_JVC']='no_data'
    
    scroll(100)

    try:
        game_caracteristique=driver.find_element(By.CLASS_NAME, "gameCharacteristicsDetailed__table")
        gaming_JVC.loc[i, 'description_JVC']=game_caracteristique.text
    except:
        gaming_JVC.loc[i, 'description_JVC']='no_data'
        


[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:01<00:00, 5.10MB/s]
100%|██████████| 262/262 [2:15:02<00:00, 30.92s/it]  


In [13]:
driver.close()

In [14]:
gaming_JVC.head(10)

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,test_note_JVC,avis_count_JVC,avis_note_JVC,description_JVC
Rank,,,,,,,,,,,,,,
2854,Catherine,PS3,2011.0,Adventure,Deep Silver,0.30,0.15,0.2,0.07,0.72,17,(145),16.8,Editeur(s) / Développeur(s) Deep Silver Atlus\...
2855,Fear Effect,PS,1998.0,Action,Eidos Interactive,0.40,0.27,0.0,0.05,0.72,17,(3),20,Editeur(s) / Développeur(s) Eidos Interactive ...
2856,FIFA Soccer 11,DS,2010.0,Sports,Electronic Arts,0.13,0.50,0.0,0.09,0.72,no_data,no_data,no_data,no_data
2857,Crysis,PC,2007.0,Shooter,Electronic Arts,0.00,0.69,0.0,0.03,0.72,18,(959),17.2,Editeur(s) / Développeur(s) Crytek Electronic ...
2858,Battlefield: Bad Company 2,PC,2010.0,Shooter,Electronic Arts,0.19,0.53,0.0,0.00,0.71,16,(186),17.4,Editeur(s) / Développeur(s) Electronic Arts Di...
2859,The Lord of the Rings: The Two Towers,XB,2002.0,Action,Electronic Arts,0.45,0.24,0.0,0.03,0.71,17,(50),17.1,Editeur(s) / Développeur(s) Griptonite Games S...
2860,Madden NFL 2003,XB,2002.0,Sports,Electronic Arts,0.67,0.02,0.0,0.03,0.71,17,(8),17.3,Editeur(s) / Développeur(s) EA Tiburon Budcat ...
2861,Dragon Quest Swords: The Masked Queen and the ...,Wii,2007.0,Role-Playing,Square Enix,0.18,0.01,0.5,0.02,0.71,13,(99),13.1,Editeur(s) / Développeur(s) Square Enix\nSorti...
2862,MLB 13: The Show,PS3,2013.0,Sports,Sony Computer Entertainment,0.66,0.00,0.0,0.05,0.71,16,(14),16.5,Editeur(s) / Développeur(s) Sony\nSortie Franc...


In [8]:
gaming_JVC.to_csv('gaming_JVC_3bis.csv')